## Import library and datasets

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import numpy as np
import re
import unicodedata
import pycountry
import pycountry_convert as pc

In [4]:
df_races = pd.read_csv('dataset/races.csv')
df_cyclists = pd.read_csv('dataset/cyclists.csv')

## Races - Syntactic and Semantic accuracy + info and descriptive statistic

In [5]:
df_races.head()

,_url,name,points,uci_points,length,climb_total,profile,startlist_quality,average_temperature,date,position,cyclist,cyclist_age,is_tarmac,is_cobbled,is_gravel,cyclist_team,delta
0,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,0,sean-kelly,22.0,True,False,False,vini-ricordi-pinarello-sidermec-1986,0.0
1,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,1,gerrie-knetemann,27.0,True,False,False,norway-1987,0.0
2,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,2,rene-bittinger,24.0,True,False,False,NaN,0.0
3,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,3,joseph-bruyere,30.0,True,False,False,navigare-blue-storm-1993,0.0
4,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,4,sven-ake-nilsson,27.0,True,False,False,spain-1991,0.0


In [6]:
df_races.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589865 entries, 0 to 589864
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   _url                 589865 non-null  object 
 1   name                 589865 non-null  object 
 2   points               589388 non-null  float64
 3   uci_points           251086 non-null  float64
 4   length               589865 non-null  float64
 5   climb_total          442820 non-null  float64
 6   profile              441671 non-null  float64
 7   startlist_quality    589865 non-null  int64  
 8   average_temperature  29933 non-null   float64
 9   date                 589865 non-null  object 
 10  position             589865 non-null  int64  
 11  cyclist              589865 non-null  object 
 12  cyclist_age          589752 non-null  float64
 13  is_tarmac            589865 non-null  bool   
 14  is_cobbled           589865 non-null  bool   
 15  is_gravel        

In [7]:
df_races.shape

(589865, 18)

In [8]:
df_races.describe()

,points,uci_points,length,climb_total,profile,startlist_quality,average_temperature,position,cyclist_age,delta
count,589388.000000,251086.000000,589865.000000,442820.000000,441671.000000,589865.000000,29933.000000,589865.000000,589752.000000,589865.000000
mean,89.221635,74.601547,166776.180584,2330.469215,2.611611,1101.161178,21.731768,74.219491,28.486208,418.292794
std,54.435330,100.947962,64545.605664,1375.710722,1.491741,380.586928,5.884761,48.404023,3.855631,842.961596
min,18.000000,6.000000,1000.000000,2.000000,1.000000,115.000000,10.000000,0.000000,13.000000,-6906.000000
25%,50.000000,16.000000,152500.000000,1309.000000,1.000000,844.000000,17.000000,32.000000,26.000000,10.000000
50%,80.000000,60.000000,178200.000000,2255.000000,2.000000,988.000000,22.000000,70.000000,28.000000,156.000000
75%,100.000000,100.000000,203500.000000,3273.000000,4.000000,1309.000000,26.000000,112.000000,31.000000,624.000000
max,350.000000,800.000000,338000.000000,6974.000000,5.000000,2047.000000,36.000000,209.000000,56.000000,61547.000000


### Duplictaes

In [9]:
df_races = df_races = pd.read_csv('dataset/races.csv')

num_duplicates = df_races.duplicated().sum()
print(f'Number of duplicate rows: {num_duplicates}')

Number of duplicate rows: 0


### _url - syntactic accuracy

In [10]:
url_pattern = re.compile(r'^[a-zA-Z0-9-]+/\d{4}/(stage-[a-zA-Z0-9-]+|prologue|results|result)$')
url_syntax_validity = df_races['_url'].apply(lambda x: bool(url_pattern.match(x)))

final_invalid_urls = df_races.loc[~url_syntax_validity, '_url']

final_invalid_url_details = final_invalid_urls.value_counts()

final_syntax_analysis_results = {
    "Total Invalid URLs": final_invalid_urls.size,
    "Sample of Invalid URLs (if any)": final_invalid_url_details.head(10).to_dict() if final_invalid_urls.size > 0 else "None"
}

final_syntax_analysis_results

{'Total Invalid URLs': 0, 'Sample of Invalid URLs (if any)': 'None'}

### _url - semantic accuracy

In [11]:
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

def extract_keywords(text):
    text = remove_accents(text)
    return set(re.findall(r'\b\w+\b', text))

url_parts = df_races['_url'].str.extract(r'(?P<event>[\w-]+)/(?P<year>\d{4})/(?P<stage>[\w-]+)')

df_races['normalized_name'] = df_races['name'].str.lower().apply(remove_accents).str.replace(' ', '-').str.replace(r'[^a-zA-Z0-9-]', '', regex=True)
df_races['event_keywords'] = url_parts['event'].str.lower().apply(remove_accents).apply(extract_keywords)
df_races['name_keywords'] = df_races['normalized_name'].apply(extract_keywords)

keyword_match = df_races.apply(lambda row: bool(row['event_keywords'] & row['name_keywords']), axis=1)

keyword_match_summary = {
    "Keyword Match Consistency": keyword_match.value_counts().to_dict()
}


In [12]:
remaining_inconsistent_rows_keyword_match = df_races.loc[~keyword_match, ['_url', 'name', 'normalized_name', 'event_keywords', 'name_keywords']].head(10)

print("Sample Rows with Remaining Event Inconsistencies (After Keyword Matching)")
display(remaining_inconsistent_rows_keyword_match) 

Sample Rows with Remaining Event Inconsistencies (After Keyword Matching)


,_url,name,normalized_name,event_keywords,name_keywords
6541,itzulia-basque-country/2007/stage-3,Vuelta al País Vasco,vuelta-al-pais-vasco,"{itzulia, basque, country}","{pais, vuelta, al, vasco}"
6542,itzulia-basque-country/2007/stage-3,Vuelta al País Vasco,vuelta-al-pais-vasco,"{itzulia, basque, country}","{pais, vuelta, al, vasco}"
6543,itzulia-basque-country/2007/stage-3,Vuelta al País Vasco,vuelta-al-pais-vasco,"{itzulia, basque, country}","{pais, vuelta, al, vasco}"
6544,itzulia-basque-country/2007/stage-3,Vuelta al País Vasco,vuelta-al-pais-vasco,"{itzulia, basque, country}","{pais, vuelta, al, vasco}"
6545,itzulia-basque-country/2007/stage-3,Vuelta al País Vasco,vuelta-al-pais-vasco,"{itzulia, basque, country}","{pais, vuelta, al, vasco}"
6546,itzulia-basque-country/2007/stage-3,Vuelta al País Vasco,vuelta-al-pais-vasco,"{itzulia, basque, country}","{pais, vuelta, al, vasco}"
6547,itzulia-basque-country/2007/stage-3,Vuelta al País Vasco,vuelta-al-pais-vasco,"{itzulia, basque, country}","{pais, vuelta, al, vasco}"
6548,itzulia-basque-country/2007/stage-3,Vuelta al País Vasco,vuelta-al-pais-vasco,"{itzulia, basque, country}","{pais, vuelta, al, vasco}"
6549,itzulia-basque-country/2007/stage-3,Vuelta al País Vasco,vuelta-al-pais-vasco,"{itzulia, basque, country}","{pais, vuelta, al, vasco}"
6550,itzulia-basque-country/2007/stage-3,Vuelta al País Vasco,vuelta-al-pais-vasco,"{itzulia, basque, country}","{pais, vuelta, al, vasco}"


In [13]:
print(len(remaining_inconsistent_rows_keyword_match))

10


In [14]:
df_races['year_from_date'] = pd.to_datetime(df_races['date'], errors='coerce').dt.year
year_match = url_parts['year'].astype(float) == df_races['year_from_date']

semantic_validity_summary = {
    "Year Consistency between _url and date": year_match.all(),
    "Year Consistency Details": year_match.value_counts().to_dict(),
}

semantic_validity_summary

{'Year Consistency between _url and date': True,
 'Year Consistency Details': {True: 589865}}

### Points - syntactic and semantic accuracy

In [15]:
points_dtype = df_races['points'].dtype
points_negative = (df_races['points'] < 0).sum()

points_analysis_summary = {
    "Data Type": points_dtype,
    "Negative Values Count": points_negative
}

points_analysis_summary

{'Data Type': dtype('float64'), 'Negative Values Count': 0}

### Uci_points - syntactic and semantic accuracy

In [16]:
points_dtype = df_races['uci_points'].dtype
points_negative = (df_races['uci_points'] < 0).sum()


points_analysis_summary = {
    "Data Type": points_dtype,
    "Negative Values Count": points_negative
}

points_analysis_summary

{'Data Type': dtype('float64'), 'Negative Values Count': 0}

### Length - syntactic and semantic accuracy

In [17]:
length_dtype = df_races['length'].dtype
length_negative = (df_races['length'] < 0).sum()


length_analysis_summary = {
    "Data Type": length_dtype,
    "Negative Values Count": length_negative,
}

length_analysis_summary


{'Data Type': dtype('float64'), 'Negative Values Count': 0}

### Climb_total - syntactic and semantic accuracy

In [18]:
climb_total_dtype = df_races['climb_total'].dtype
climb_total_negative = (df_races['climb_total'] < 0).sum()


climb_total_analysis_summary = {
    "Data Type": climb_total_dtype,
    "Negative Values Count": climb_total_negative
}

climb_total_analysis_summary



{'Data Type': dtype('float64'), 'Negative Values Count': 0}

### Ensure the climb_total increases with the profile

Given that the profile says how mountainous the stage's terrain is (the five profile values probably correspond to the 5 icons explained here: https://www.procyclingstats.com/info/profile-score-explained), we'd expect higher values of profile to have higher average climb_totals. This is indeed what we observe.

In [19]:
average_climb_total_per_profile = df_races.groupby('profile')['climb_total'].mean().reset_index()

average_climb_total_per_profile

,profile,climb_total
0,1.0,1115.032447
1,2.0,2216.014574
2,3.0,2417.451732
3,4.0,3493.641104
4,5.0,3737.367327


### Profile - syntactic and semantic accuracy

In [20]:
profile_dtype = df_races['profile'].dtype
profile_non_integer = df_races['profile'].dropna().apply(lambda x: not float(x).is_integer()).sum()

profile_out_of_range = df_races['profile'].dropna().apply(lambda x: x < 1 or x > 5).sum()

profile_analysis_summary = {
    "Data Type": profile_dtype,
    "Non-Integer Values Count": profile_non_integer,
    "Values Out of Range (1-5)": profile_out_of_range,
}

profile_analysis_summary



{'Data Type': dtype('float64'),
 'Non-Integer Values Count': 0,
 'Values Out of Range (1-5)': 0}

### Startlist_quality - syntactic and semantic accuracy

In [21]:
sl_dtype = df_races['startlist_quality'].dtype
sl_negative = (df_races['startlist_quality'] < 0).sum()


sl_analysis_summary = {
    "Data Type": sl_dtype,
    "Negative Values Count": sl_negative
}

sl_analysis_summary

{'Data Type': dtype('int64'), 'Negative Values Count': 0}

### Make sure the startlist_quality is realistic

According to procycliststats, the theoretical maximum limit of startlist_quality for a 150-person race is 2275 points (https://www.procyclingstats.com/calendar/uci/startlist-quality)

In [22]:
df_races['startlist_quality_issue'] = ~df_races['startlist_quality'].between(0, 2275, inclusive="both").fillna(False)

invalid_startlist_quality_records = df_races[df_races['startlist_quality_issue'] == True]
print(invalid_startlist_quality_records)

Empty DataFrame
Columns: [_url, name, points, uci_points, length, climb_total, profile, startlist_quality, average_temperature, date, position, cyclist, cyclist_age, is_tarmac, is_cobbled, is_gravel, cyclist_team, delta, normalized_name, event_keywords, name_keywords, year_from_date, startlist_quality_issue]
Index: []

[0 rows x 23 columns]


### Average_temperature - syntactic and semantic accuracy

In [23]:
temperature_dtype = df_races['average_temperature'].dtype
temperature_negative = (df_races['average_temperature'] < 0).sum()

temperature_analysis_summary = {
    "Data Type": temperature_dtype,
    "Negative Values Count": temperature_negative,
}

temperature_analysis_summary

{'Data Type': dtype('float64'), 'Negative Values Count': 0}

In [24]:
df_races['average_temperature_issue'] = ~df_races['average_temperature'].between(-43, 43, inclusive="both").fillna(False)

invalid_temperature_records = df_races[df_races['average_temperature'] == True]
print(invalid_temperature_records)

Empty DataFrame
Columns: [_url, name, points, uci_points, length, climb_total, profile, startlist_quality, average_temperature, date, position, cyclist, cyclist_age, is_tarmac, is_cobbled, is_gravel, cyclist_team, delta, normalized_name, event_keywords, name_keywords, year_from_date, startlist_quality_issue, average_temperature_issue]
Index: []

[0 rows x 24 columns]


### Date - syntactic and semantic accuracy

La coerenza dell'anno è stata fatta analizzando _url, non importa farla nuovamente

In [25]:
date_validity = pd.to_datetime(df_races['date'], errors='coerce').notnull().all() 
date_parsed = pd.to_datetime(df_races['date'], errors='coerce')  

future_dates = (date_parsed > pd.Timestamp.now()).sum() 

date_analysis_summary = {
    "All Dates Valid": date_validity,
    "Count of Future Dates": future_dates,
}

date_analysis_summary


{'All Dates Valid': True, 'Count of Future Dates': 0}

### Position - syntactic accuracy

In [26]:
position_dtype = df_races['position'].dtype
position_negative = (df_races['position'] < 0).sum()

position_analysis_summary = {
    "Data Type": position_dtype,
    "Negative Values Count": position_negative
}

position_analysis_summary

{'Data Type': dtype('int64'), 'Negative Values Count': 0}

### Position and date - semantic accuracy

In [27]:
url_parts = df_races['_url'].str.extract(r'(?P<event>[\w-]+)/(?P<year>\d{4})/(?P<stage>[\w-]+)')
df_races['event'] = url_parts['event']
df_races['year'] = url_parts['year'].astype(float, errors='ignore')
df_races['stage'] = url_parts['stage']

df_races['date_parsed'] = date_parsed

grouped_data = df_races.sort_values(by=['event', 'year', 'stage', 'date_parsed'])

# Sorting by position within each group before checking
chronological_order_check = grouped_data.groupby(['event', 'year', 'stage']).apply(
    lambda df: df['date_parsed'].is_monotonic_increasing
).all()

# Sorting by position before checking
sorted_by_position = grouped_data.sort_values(by=['event', 'year', 'stage', 'position'])
position_order_check = sorted_by_position.groupby(['event', 'year', 'stage']).apply(
    lambda df: df['position'].is_monotonic_increasing
).all()

semantic_order_summary = {
    "Dates in Chronological Order for All Groups": chronological_order_check,
    "Positions in Ascending Order for All Groups": position_order_check
}

semantic_order_summary


/tmp/ipykernel_18832/3636021410.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  chronological_order_check = grouped_data.groupby(['event', 'year', 'stage']).apply(
/tmp/ipykernel_18832/3636021410.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  position_order_check = sorted_by_position.groupby(['event', 'year', 'stage']).apply(


{'Dates in Chronological Order for All Groups': True,
 'Positions in Ascending Order for All Groups': True}

### is_tarmac/is_cobbled/is_gravel - syntactic and semantic accuracy

In [28]:
syntactic_summary = {
    "is_tarmac Type": df_races['is_tarmac'].dtype,
    "is_cobbled Type": df_races['is_cobbled'].dtype,
    "is_gravel Type": df_races['is_gravel'].dtype,
}
inconsistent_rows = df_races[(df_races['is_tarmac'] == True) & 
                             (df_races['is_cobbled'] == True) & 
                             (df_races['is_gravel'] == True)]

at_least_one_true = df_races[['is_tarmac', 'is_cobbled', 'is_gravel']].any(axis=1).all()

semantic_summary = {
    "Total Rows with All Three Surfaces as True": inconsistent_rows.shape[0],
    "At Least One Surface True for All Rows": at_least_one_true
}

# Combining results
analysis_summary = {
    "Syntactic Summary": syntactic_summary,
    "Semantic Summary": semantic_summary
}

analysis_summary

{'Syntactic Summary': {'is_tarmac Type': dtype('bool'),
  'is_cobbled Type': dtype('bool'),
  'is_gravel Type': dtype('bool')},
 'Semantic Summary': {'Total Rows with All Three Surfaces as True': 0,
  'At Least One Surface True for All Rows': False}}

I risultati riguardo la presenza di almeno un valore impostato a True danno esito negativo però possono essere visti come "terreno misto"?

### cyclist_team - syntactic accuracy -> not very useful

### cyclist and cyclist age - syntactic and semantic accuracy -> wait until the analysis on cyclists dataset

### delta - syntactic and semantic accuracy

In [29]:
delta_dtype = df_races['delta'].dtype
delta_negative = (df_races['delta'] < 0).sum()


delta_analysis_summary = {
    "Data Type": delta_dtype,
    "Negative Values Count": delta_negative,
}

delta_analysis_summary

{'Data Type': dtype('float64'), 'Negative Values Count': 86}

In [30]:
delta_zeroes = (df_races['delta'] == 0).sum()

da imputare a partecipanti penalizzati/squalificati?

In [31]:
df_races['date_parsed'] = pd.to_datetime(df_races['date'], errors='coerce')

df_races = df_races.sort_values(by=['event', 'year', 'stage', 'date_parsed'])
df_races['time_difference'] = df_races.groupby(['event', 'year', 'stage'])['date_parsed'].diff().dt.total_seconds()

tolerance = 10  # seconds
approximate_matches = (df_races['delta'].notnull()) & (df_races['time_difference'].notnull()) & (
    abs(df_races['delta'] - df_races['time_difference']) <= tolerance
)

approximate_match_count = approximate_matches.sum()
total_comparable_rows = df_races['delta'].notnull().sum()
mismatched_rows = df_races[~approximate_matches & df_races['delta'].notnull() & df_races['time_difference'].notnull()]

match_summary = {
    "Approximate Matches Count": approximate_match_count,
    "Total Comparable Rows": total_comparable_rows,
    "Approximate Match Percentage": (approximate_match_count / total_comparable_rows) * 100 if total_comparable_rows > 0 else 0
}

match_summary



{'Approximate Matches Count': 147475,
 'Total Comparable Rows': 589865,
 'Approximate Match Percentage': 25.001483390267264}

#### Fixing the delta

In [32]:
df_races.loc[~approximate_matches & df_races['delta'].notnull() & df_races['time_difference'].notnull(), 'delta'] = df_races['time_difference']

#### Redo th check

In [33]:
df_races['date_parsed'] = pd.to_datetime(df_races['date'], errors='coerce')

df_races = df_races.sort_values(by=['event', 'year', 'stage', 'date_parsed'])
df_races['time_difference'] = df_races.groupby(['event', 'year', 'stage'])['date_parsed'].diff().dt.total_seconds()

tolerance = 10  # seconds
approximate_matches = (df_races['delta'].notnull()) & (df_races['time_difference'].notnull()) & (
    abs(df_races['delta'] - df_races['time_difference']) <= tolerance
)

approximate_match_count = approximate_matches.sum()
total_comparable_rows = df_races['delta'].notnull().sum()
mismatched_rows = df_races[~approximate_matches & df_races['delta'].notnull() & df_races['time_difference'].notnull()]

match_summary = {
    "Approximate Matches Count": approximate_match_count,
    "Total Comparable Rows": total_comparable_rows,
    "Approximate Match Percentage": (approximate_match_count / total_comparable_rows) * 100 if total_comparable_rows > 0 else 0
}

match_summary

{'Approximate Matches Count': 584584,
 'Total Comparable Rows': 589865,
 'Approximate Match Percentage': 99.10471039983724}

## Cyclists - Syntactic and Semantic accuracy + info and descriptive statistics

In [34]:
df_cyclists.head()

,_url,name,birth_year,weight,height,nationality
0,bruno-surra,Bruno Surra,1964.0,NaN,NaN,Italy
1,gerard-rue,Gérard Rué,1965.0,74.0,182.0,France
2,jan-maas,Jan Maas,1996.0,69.0,189.0,Netherlands
3,nathan-van-hooydonck,Nathan Van Hooydonck,1995.0,78.0,192.0,Belgium
4,jose-felix-parra,José Félix Parra,1997.0,55.0,171.0,Spain


In [35]:
df_cyclists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6134 entries, 0 to 6133
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   _url         6134 non-null   object 
 1   name         6134 non-null   object 
 2   birth_year   6121 non-null   float64
 3   weight       3078 non-null   float64
 4   height       3143 non-null   float64
 5   nationality  6133 non-null   object 
dtypes: float64(3), object(3)
memory usage: 287.7+ KB


In [36]:
df_cyclists.shape

(6134, 6)

In [37]:
df_cyclists.describe()

,birth_year,weight,height
count,6121.000000,3078.000000,3143.000000
mean,1974.071884,68.658739,179.815145
std,15.535834,6.348183,6.443447
min,1933.000000,48.000000,154.000000
25%,1962.000000,64.000000,175.000000
50%,1974.000000,69.000000,180.000000
75%,1987.000000,73.000000,184.000000
max,2004.000000,94.000000,204.000000


### Duplicates

In [38]:
num_duplicates = df_cyclists.duplicated().sum()
print(f'Number of duplicate rows: {num_duplicates}')

Number of duplicate rows: 0


### birth_year - syntactic and semantic accuracy

In [39]:
df_cyclists['birth_year'] = pd.to_numeric(df_cyclists['birth_year'], errors='coerce').astype('Int64')

min_valid_year = df_cyclists['birth_year'].min()
max_valid_year = df_cyclists['birth_year'].max()

valid_years = df_cyclists['birth_year'].between(min_valid_year, max_valid_year, inclusive='both')
invalid_birth_years = df_cyclists[~valid_years]

# Count of valid and invalid years
valid_years_count = valid_years.sum()
total_rows = df_cyclists.shape[0]

invalid_birth_years = df_cyclists[~df_cyclists['birth_year'].between(min_valid_year, max_valid_year, inclusive='both')]
print(invalid_birth_years)

# Summary of results
year_summary = {
    "Total Rows": total_rows,
    "Valid Birth Years Count": valid_years_count,
    "Invalid Birth Years Count": total_rows - valid_years_count
}

print(year_summary)

print("Missing values:" ,df_cyclists['birth_year'].isnull().sum())



Empty DataFrame
Columns: [_url, name, birth_year, weight, height, nationality]
Index: []
{'Total Rows': 6134, 'Valid Birth Years Count': 6121, 'Invalid Birth Years Count': 13}
Missing values: 13


### weight and height - syntactic and semantic accuracy

In [40]:

min_valid_weight = df_cyclists['weight'].min() 
max_valid_weight = df_cyclists['weight'].max()
min_valid_height = df_cyclists['height'].min()  
max_valid_height = df_cyclists['height'].max()  

invalid_weight = df_cyclists[~df_cyclists['weight'].between(min_valid_weight, max_valid_weight, inclusive='both')]
invalid_height = df_cyclists[~df_cyclists['height'].between(min_valid_height, max_valid_height, inclusive='both')]

weight_summary = {
    "Valid Weight Count": df_cyclists['weight'].between(min_valid_weight, max_valid_weight, inclusive='both').sum(),
    "Invalid Weight Count": invalid_weight.shape[0]
}

height_summary = {
    "Valid Height Count": df_cyclists['height'].between(min_valid_height, max_valid_height, inclusive='both').sum(),
    "Invalid Height Count": invalid_height.shape[0]
}

print({"Weight Summary": weight_summary})
print({"Height Summary": height_summary})

print("Missing values:" ,df_cyclists['weight'].isnull().sum())
print("Missing values:" ,df_cyclists['height'].isnull().sum())


{'Weight Summary': {'Valid Weight Count': 3078, 'Invalid Weight Count': 3056}}
{'Height Summary': {'Valid Height Count': 3143, 'Invalid Height Count': 2991}}
Missing values: 3056
Missing values: 2991


### nationality - syntactic and semantic accuracy

In [41]:
nationality_type = df_cyclists['nationality'].dtype
unique_nationalities = df_cyclists['nationality'].unique()

nationality_summary = {
    "Data Type": nationality_type,
    "Unique Nationalities Count": len(unique_nationalities),
}

print(nationality_summary)

df_cyclists = pd.read_csv('dataset/cyclists.csv')

# Helper function to standardize country names
def standardize_country_name(country_name):
    if pd.isna(country_name):  # Check for NaN (missing) values
        return None
    try:
        # Try to get the alpha-2 code from the name
        country_code = pc.country_name_to_country_alpha2(country_name, cn_name_format="default")
        # Get the country name from the alpha-2 code
        return pycountry.countries.get(alpha_2=country_code).name
    except KeyError:
        # Return None if no match is found
        return None

# Apply the standardization function and validate
df_cyclists['standardized_nationality'] = df_cyclists['nationality'].apply(standardize_country_name)
df_cyclists['valid_nationality'] = df_cyclists['standardized_nationality'].notna()

# Filter and output records where valid_nationality is False
invalid_nationality_records = df_cyclists[df_cyclists['valid_nationality'] == False]
print(invalid_nationality_records[['name', 'nationality', 'standardized_nationality', 'valid_nationality']])


{'Data Type': dtype('O'), 'Unique Nationalities Count': 73}
               name nationality standardized_nationality  valid_nationality
9     Scott  Davies         NaN                     None              False
102   Primož  Čerin  Yugoslavia                     None              False
6100   Kam-Po  Wong    Hongkong                     None              False


### cyclist (see races dataset section) and name (cyclists dataset section) - syntactic and semantic accuracy

In [42]:
unique_race_cyclists = df_races['cyclist'].unique()

missing_cyclists = [cyclist for cyclist in unique_race_cyclists if cyclist not in df_cyclists['_url'].values]

cyclist_check_summary = {
    "Total Unique Cyclists in Races": len(unique_race_cyclists),
    "Missing Cyclists Count": len(missing_cyclists),
    "All Cyclists Present": len(missing_cyclists) == 0
}

cyclist_check_summary



{'Total Unique Cyclists in Races': 6095,
 'Missing Cyclists Count': 0,
 'All Cyclists Present': True}

In [43]:
unique_cyclist_urls = df_cyclists['_url'].unique()
missing_cyclists_in_races = [cyclist for cyclist in unique_cyclist_urls if cyclist not in df_races['cyclist'].values]

reverse_cyclist_check_summary = {
    "Total Unique Cyclists in Cyclists Dataset": len(unique_cyclist_urls),
    "Missing Cyclists in Races Count": len(missing_cyclists_in_races),
    "All Cyclists in Races": len(missing_cyclists_in_races) == 0
}

reverse_cyclist_check_summary

{'Total Unique Cyclists in Cyclists Dataset': 6134,
 'Missing Cyclists in Races Count': 39,
 'All Cyclists in Races': False}

In [44]:
# Get the unique cyclist URLs from the races dataframe
unique_race_cyclists = df_races['cyclist'].unique()

# Filter the cyclists dataframe to keep only rows where the cyclist is present in the races dataframe
df_cyclists = df_cyclists[df_cyclists['_url'].isin(unique_race_cyclists)]

# Display the updated cyclists dataframe
df_cyclists.head()

,_url,name,birth_year,weight,height,nationality,standardized_nationality,valid_nationality
0,bruno-surra,Bruno Surra,1964.0,NaN,NaN,Italy,Italy,True
1,gerard-rue,Gérard Rué,1965.0,74.0,182.0,France,France,True
2,jan-maas,Jan Maas,1996.0,69.0,189.0,Netherlands,Netherlands,True
3,nathan-van-hooydonck,Nathan Van Hooydonck,1995.0,78.0,192.0,Belgium,Belgium,True
4,jose-felix-parra,José Félix Parra,1997.0,55.0,171.0,Spain,Spain,True


In [45]:
unique_cyclist_urls = df_cyclists['_url'].unique()
missing_cyclists_in_races = [cyclist for cyclist in unique_cyclist_urls if cyclist not in df_races['cyclist'].values]

reverse_cyclist_check_summary = {
    "Total Unique Cyclists in Cyclists Dataset": len(unique_cyclist_urls),
    "Missing Cyclists in Races Count": len(missing_cyclists_in_races),
    "All Cyclists in Races": len(missing_cyclists_in_races) == 0
}

reverse_cyclist_check_summary

{'Total Unique Cyclists in Cyclists Dataset': 6095,
 'Missing Cyclists in Races Count': 0,
 'All Cyclists in Races': True}

### cyclist_age (see races dataset section)

In [46]:
df_cyclists['birth_year'] = pd.to_numeric(df_cyclists['birth_year'], errors='coerce')
df_races['date_parsed'] = pd.to_datetime(df_races['date'], errors='coerce')
df_races['race_year'] = df_races['date_parsed'].dt.year

df_merged = df_races.merge(df_cyclists, left_on='cyclist', right_on='_url', how='left')

df_merged['calculated_age'] = df_merged['race_year'] - df_merged['birth_year']

df_merged['age_difference'] = df_merged['calculated_age'] - df_merged['cyclist_age']

age_mismatch = df_merged[df_merged['age_difference'].notnull() & (df_merged['age_difference'] != 0)]

age_mismatch_summary = {
    "Total Rows Compared": df_merged['age_difference'].notnull().sum(),
    "Mismatched Age Count": age_mismatch.shape[0]
}

age_mismatch_summary

{'Total Rows Compared': 589752, 'Mismatched Age Count': 0}

#### Extracting the correct dataset with fixed delta is races

In [47]:
# Races
df_races.to_csv('df_races_cleaned.csv', index=False)

# Cyclists
df_cyclists.to_csv('df_cyclists_cleaned.csv', index=False)